In [ ]:
import pyro
import pyro.distributions as dist
from pyro.distributions import constraints
from pyro.infer import NUTS, MCMC, Predictive
from pyro.optim import Adam
import pandas as pd
import torch
from tqdm import tqdm
import us
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.ticker as ticker
import numpy as np
import re

In [ ]:
presidential_forecast = pd.read_csv('../data/presidential_forecast/presidential_forecast.tsv', sep='\s*\t')
presidential_forecast

In [ ]:
region_info = pd.read_csv('../data/presidential_forecast/region_info.tsv', sep='\s*\t')
region_info

In [ ]:
combined = pd.merge(presidential_forecast, region_info, on='state')
combined = combined.dropna()
combined

In [ ]:
years = combined['year'].unique()
regions = combined['region'].unique()
num_years = len(years)
num_regions = len(regions)
year_map = {year: i for i, year in enumerate(sorted(years))}
region_map = {region: i for i, region in enumerate(sorted(regions))}
(num_years, num_regions, year_map, region_map)

In [ ]:
tcombined = [{k: torch.from_numpy(np.vectorize(lambda x: year_map.get(x, region_map.get(x, x)))(v.values))
             for k, v in combined[combined['year'] == year].iteritems()} for year in sorted(years)]
tcombined

In [ ]:
train_data = tcombined[:-1]
test_data = tcombined[-1:]

In [ ]:
def model(year_data):
    # TODO: Write model

In [ ]:
samples = ... # TODO Run NUTS and replace ... with the samples

In [ ]:
predictive = Predictive(model, {k: samples[k] for k in {'beta', 'sigma', 'tau_gamma', 'tau_delta'}})
predicted_y = predictive.forward([{k: v for k, v in dt.items() if k != 'Dvote'} for dt in test_data])['y_11']
predicted_y_mean = predicted_y.mean(axis=0)
predicted_y_var = predicted_y.var(axis=0)
(predicted_y_mean, predicted_y_var, (predicted_y_mean - test_data[0]['Dvote']).abs())

In [ ]:
plt.violinplot(predicted_y.transpose(0, 1))
plt.show()